In [1]:
from os.path import join
import numpy as np
import pandas as pd 
import gc
from tqdm import tqdm
import cv2
import time
import datetime
import yaml
import os
import random
import sys

import timm
import ttach as tta
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

from sklearn.metrics import roc_auc_score

from dataset import MelanomaDataset, train_transforms, test_transforms, tta_transforms

import warnings
warnings.filterwarnings('ignore')

In [2]:
exp_name = 'exp_train_07'
with open(join('../configs/', f'{exp_name}.yaml'), 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

print('Config params:')
print(config)

Config params:
{'INPUT_DIR': '../input/', 'MODELS_DIR': '../models/', 'SUBMISSIONS_DIR': '../submissions/', 'batch_size': 10, 'learning_rate': 0.0003, 'n_epochs': 25, 'n_workers': 8, 'early_stopping_patience': 5, 'reduce_lr_on_plateau_patience': 1, 'reduce_lr_on_plateau_factor': 0.2, 'weight_decay': 0.001, 'n_folds': 5, 'images_size': '512x512', 'model_name': 'tf_efficientnet_b3_ns', 'exp_train_name': 'exp_train_05', 'use_test_labels': False, 'seed': 42, 'device': 'cuda:0', 'folds_train_file': 'train_folds_stratified'}


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True

seed_everything(config['seed'])
INPUT_DIR = config['INPUT_DIR']
MODELS_DIR = config['MODELS_DIR']
SUBMISSIONS_DIR = config['SUBMISSIONS_DIR']
os.makedirs(join(MODELS_DIR, exp_name), exist_ok=True)
log_file = join(MODELS_DIR, exp_name, f'{exp_name}.txt')
try: os.mknod(log_file)
except: pass

BS = config['batch_size']
LR = config['learning_rate']
EPOCHS = config['n_epochs']
WORKERS = config['n_workers']
ES_PATIENCE = config['early_stopping_patience']
REDUCELR_PATIENCE = config['reduce_lr_on_plateau_patience']
REDUCELR_FACTOR = config['reduce_lr_on_plateau_factor']
WEIGHT_DECAY = config['weight_decay']
# STEP_SIZE = config['steplr_step_size']

N_FOLDS = config['n_folds']
images_size = config['images_size']
model_name = config['model_name'] # https://github.com/rwightman/pytorch-image-models
device = config['device'] # torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()
gc.collect();

In [ ]:
df_train = pd.read_csv(join(INPUT_DIR, f'{config["folds_train_file"]}.csv'))
df_test = pd.read_csv(join(INPUT_DIR, 'sample_submission.csv'))
test = MelanomaDataset(df_test, INPUT_DIR, images_size, test_transforms)

In [ ]:
preds = torch.zeros((len(test), 1), dtype=torch.float32, device=device)

folds = list(range(N_FOLDS))
for fold in folds:
    print(f'Fold: {fold}')
    folds_to_train = list(set(folds)-set([fold]))
    
    best_val = None 
    patience = ES_PATIENCE
    model = timm.create_model(model_name, pretrained=True, num_classes=1)
    model.cuda()
    
    optim = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, amsgrad=True)
    # scheduler = StepLR(optim, step_size=STEP_SIZE, gamma=0.3)
    scheduler = ReduceLROnPlateau(optim, patience=REDUCELR_PATIENCE, factor=REDUCELR_FACTOR, 
                                  mode='max', verbose=True)
    criterion = nn.BCEWithLogitsLoss()

    train_df = df_train[df_train['kfold'].isin(folds_to_train)].reset_index(drop=True)
    valid_df = df_train[df_train['kfold'] == fold].reset_index(drop=True)

    train = MelanomaDataset(train_df, INPUT_DIR, images_size, train_transforms)
    val = MelanomaDataset(valid_df, INPUT_DIR, images_size, test_transforms)
    
    train_loader = DataLoader(train, batch_size=BS, shuffle=True, num_workers=WORKERS)
    val_loader = DataLoader(val, batch_size=BS, shuffle=False, num_workers=WORKERS)

    model_path = join(MODELS_DIR, exp_name, f'fold_{fold}_weight.pth')

    for epoch in range(EPOCHS):
        start_time = time.time()
        epoch_loss = 0
        
        model.train()
        for i, (x, y) in enumerate(tqdm(train_loader, total=len(train_loader), position=0, leave=True)):
            x = torch.tensor(x, device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            optim.zero_grad()
            z = model(x)
            loss = criterion(z, y.unsqueeze(1))
            loss.backward()
            optim.step()
            epoch_loss += loss.item()

        model.eval()
        val_preds = torch.zeros((len(valid_df), 1), dtype=torch.float32, device=device)
        with torch.no_grad():
            for j, (x_val, y_val) in enumerate(tqdm(val_loader, total=len(val_loader), position=0, leave=True)):
                x_val = torch.tensor(x_val, device=device, dtype=torch.float32)
                y_val = torch.tensor(y_val, device=device, dtype=torch.float32)
                z_val = model(x_val)
                val_pred = torch.sigmoid(z_val)
                val_preds[j*x_val.shape[0]:j*x_val.shape[0] + x_val.shape[0]] = val_pred
            val_auc = roc_auc_score(valid_df['target'].values, val_preds.cpu().detach())
            
            print('Epoch {:02}: | Loss: {:.4f} | Val roc_auc: {:.4f} | Training time: {}'.format(
            epoch+1, 
            epoch_loss, 
            val_auc, 
            str(datetime.timedelta(seconds=time.time() - start_time))[:7]))
            
            scheduler.step(val_auc)
            # During the first iteration (first epoch) best validation is set to None
            if not best_val:
                best_val = val_auc
                torch.save(model, model_path) 
                continue
                
            if val_auc >= best_val:
                best_val = val_auc
                patience = ES_PATIENCE
                torch.save(model, model_path)
            else:
                patience -= 1
                if patience == 0:
                    print('Early stopping. Best Val roc_auc: {:.4f}'.format(best_val))
                    break
    
    test_loader = DataLoader(test, batch_size=BS, shuffle=False, num_workers=WORKERS)
    model = torch.load(model_path)
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
    with torch.no_grad():
        for i, (x_test, _) in enumerate(tqdm(test_loader, total=len(test_loader), position=0, leave=True)):
            x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
            z_test = tta_model(x_test)
            z_test = torch.sigmoid(z_test)
            preds[i*x_test.shape[0]:i*x_test.shape[0] + x_test.shape[0]] += z_test

    del train, val, train_loader, val_loader, train_df, valid_df, x, y, x_val, y_val, x_test, _
    torch.cuda.empty_cache()
    gc.collect()

preds /= len(folds)

In [ ]:
print('Making submission...')
sub = pd.read_csv(join(INPUT_DIR, 'sample_submission.csv'))
sub['target'] = preds.cpu().detach().numpy().reshape(-1,)
sub.to_csv(join(SUBMISSIONS_DIR, f'{exp_name}.csv'), index=False)
torch.cuda.empty_cache()
gc.collect()
print('Submission is created...')

In [4]:
print('Getting result on hold-out set...')

df = pd.read_csv(join(INPUT_DIR, f'{config["folds_train_file"]}.csv'))
df_hold_out = df[df['kfold'] == -1].reset_index(drop=True)
hold_out = MelanomaDataset(df_hold_out, INPUT_DIR, images_size, test_transforms)

preds = torch.zeros((len(hold_out), 1), dtype=torch.float32, device=device)

folds = list(range(N_FOLDS))
for fold in folds:
    print(f'{fold} fold model:')
    hold_out_loader = DataLoader(hold_out, batch_size=BS, shuffle=False, num_workers=WORKERS)

    model_path = join(MODELS_DIR, exp_name, f'fold_{fold}_weight.pth')
    model = torch.load(model_path)
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
    with torch.no_grad():
        for i, (x_test, _) in enumerate(tqdm(hold_out_loader, total=len(hold_out_loader), position=0, leave=True)):
            x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
            z_test = tta_model(x_test)
            z_test = torch.sigmoid(z_test)
            preds[i*x_test.shape[0]:i*x_test.shape[0] + x_test.shape[0]] += z_test
preds /= len(folds)
preds = preds.cpu().detach().numpy()

df_hold_out_pred = df_hold_out[['image_name', 'target']]
df_hold_out_pred['prediction'] = preds.reshape(-1, ).tolist()
df_hold_out_pred.to_csv(join(MODELS_DIR, exp_name, f'{exp_name}_hold_out.csv'), index=False)

hold_out_auc = roc_auc_score(df_hold_out['target'].values, preds)
os.mknod(join(MODELS_DIR, exp_name, f'roc-auc:{hold_out_auc:.4f}'))
print(f'\nROC AUC on hold-out set: {hold_out_auc:.4f}')
torch.cuda.empty_cache()
gc.collect()

Getting result on hold-out set...


  0%|          | 0/332 [00:00<?, ?it/s]

0 fold model:


  0%|          | 0/332 [00:00<?, ?it/s]

1 fold model:


...

3 fold model:


  0%|          | 0/332 [00:00<?, ?it/s]

4 fold model:


100%|██████████| 332/332 [02:34<00:00,  2.15it/s]


ROC AUC on hold-out set: 0.8529
